# dEdx correction fits

In [ ]:
from RootInteractive.Tools.aliTreePlayer import *
from TTreeHnInteractive.TTreeHnBrowser import *
#from InteractiveDrawing.bokeh.bokehTools import *
#from MLpipeline.NDFunctionInterface import  DataContainer, Fitter
#from InteractiveDrawing.bokeh.bokehDrawPanda import *
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.MLpipeline.NDFunctionInterface import  DataContainer, Fitter
from pprint import pprint
import uproot
output_notebook()
p3 = figure(plot_width=700, plot_height=400, title="template")
c1 = ROOT.TCanvas("test","test",800,500)

In [ ]:
ROOT.TFile.SetCacheFileDir("../data/")
treeMap, treeList, fileList = LoadTrees("cat map.list",".*",".*_Tgl_.*",".*",0)
#treeMap = ROOT.AliTreePlayer.LoadTrees("cat map.list",".*",".*_Tgl_.*",".*","","")
treeToAnyTree(treeMap)
#print(RenderTree(treeMap.anyTree)) # enable this line to visualizae tree stutures (tree,friends,branches aliases)

In [ ]:
#print(RenderTree(treeMap.anyTree))

## Select variables of interest

In [ ]:
varMedian=[".*Center"]
print(findSelectedBranches(treeMap,varMedian,exclude=[".*LHC.*",".*log.*"], columnMask=[["Center",""]]))
dfVariables=tree2Panda(treeMap,[".*Center.*"],"entries>=0",exclude=[".*LHC.*",".*log.*"], columnMask=[["Center",""]])
varList=dfVariables.columns.values
print("Variable list:", varList)
dfVariables.head(2)

## Select variables to fit

In [ ]:
dfdEdx=tree2Panda(treeMap,[".*hdEdxAShift.*binMedian"],"entries>=0",columnMask=[["_pass1_hdEdxAShifttMNTglDist_binMedian",""]])
dfdEdxList=dfdEdx.columns.values
#dfdEdx=pd.concat([dfdEdx, dfVariables],axis=1)
print(dfdEdxList)
dfdEdx.head(5)
dfStat=tree2Panda(treeMap,[".*hdEdxAShift.*entries"],"entries>=0",exclude=[".*entriesG.*"],columnMask=[["_pass1_hdEdxAShifttMNTglDist_",""]])
for a in dfStat.columns.values:
    dfStat[a+"S"]=1/np.sqrt(1+dfStat[a])

In [ ]:
df=pd.concat([dfdEdx, dfVariables,dfStat],axis=1)
df.head(5)

In [ ]:
df=pd.concat([dfdEdx, dfVariables,dfStat],axis=1)
npoints=df.shape[0]
#fitter.Fit()

for varFit in dfdEdxList:
    dataContainer = DataContainer(df,  varList, varFit, [npoints//2,npoints//2]) 
    fitter = Fitter(dataContainer)
    fitter.Register_Method('RF','RandomForest', 'Regressor', n_estimators=100, max_depth=10)
    fitter.Register_Method('RF200','RandomForest', 'Regressor', n_estimators=200, max_depth=10)
    fitter.Fit()
    fitter.AppendStatPandas("RF",df,varFit)
    fitter.AppendStatPandas("RF200",df,varFit)
    df[varFit+'RFPull']=(df[varFit]-df[varFit+"RFMean"])/df[varFit+"RFRMS"]
    
#test=dataContainer.Test_sample

In [ ]:
df.head(5)

## Create dashboards

In [ ]:
tooltips=[ (a,"@"+a) for a in varList ]
tooltips+=[ (a,"@"+a) for a in dfdEdxList]
figureArray=[ [[a+"RFMedian"],[a],{"colorZvar":a+"RFRMS"}] for a in dfdEdxList ]
figureArray+=[ [[a+"RFMedian"],[a+"RFRMS"],{"colorZvar":a+"entriesS"}] for a in dfdEdxList ]
figureArray+=[["table"]]
#
figureArrayD=[ [[a+"RFMedian"],[a+"-"+a+"RFMedian"],{"colorZvar":a+"RFRMS"}] for a in dfdEdxList ]
figureArrayD+=[ [[a+"RFMedian"],[a+"RFPull"],{"colorZvar":a+"entriesS"}] for a in dfdEdxList ]
figureArrayD+=[["table"]]
#pprint(figureArray)
#
figureArrayD2=[ [[a+"RFMedian"],[a+"RF200Median"+"-"+a+"RFMedian"],{"colorZvar":a+"RFRMS"}] for a in dfdEdxList ]
figureArrayD2+=[ [[a+"RFMedian"],[a+"RFPull"],{"colorZvar":a+"entriesS"}] for a in dfdEdxList ]
figureArrayD2+=[["table"]]
#pprint(figureArray)

#
wIndex=0
widgets="query.custom(),"
widgetList=[ ['range',[a]] for a in varList ]
widgetLayout=[]
index=0
for a in dfStat.columns.values: 
    if "entriesS" in a:
        widgetList.append(['range',[a],{'bins':30}])
#pprint(figureArray)        
#pprint(widgetList)
pprint(figureArrayD) 

In [ ]:
#df['LHC15oRFRMS'].plot.hist(bins=100, logy=1)
df['LHC15oRFPull'].plot.hist(bins=100,logy=1)

In [ ]:
#widgets="query.custom(),slider.shiftM(-150,150,15,-150,150)"
figureLayout: str = '((0,1,2),(3,4,5,commonY=3),(6, commonX=4,plot_height=150),commonY=1, commonX=0, x_visible=1,y_visible=0,plot_height=300,plot_width=1300)'
widgetLayout='((0,1,2,3),(4,5,6))'    
fig=bokehDrawSA.fromArray(df, "LHC15o!=0&LHC18qentriesS<0.1", figureArray,widgetList,tooltips=tooltips, layout=figureLayout,widgetLayout=widgetLayout) 

In [ ]:
output_file("dEdxCorrectionFit_Delta.html")
figureLayout: str = '((0,1,2),(3,4,5,commonY=3),(6, commonX=4,plot_height=150),commonY=1, commonX=0, x_visible=1,y_visible=0,plot_height=300,plot_width=1300)'
widgetLayout='((0,1,2,3),(4,5,6))'
fig=bokehDrawSA.fromArray(df, "LHC15o!=0&LHC18rentriesS<0.1&LHC18qentriesS<0.10", figureArrayD,widgetList,tooltips=tooltips, layout=figureLayout,widgetLayout=widgetLayout)

In [ ]:
drawColzArray(df,"LHC15o!=0&LHC15oentriesS<0.05","LHC15o", "LHC15oRFMean","LHC15oentriesS",None,doDraw=1)

In [ ]:
drawColzArray(df,"LHC15o!=0&LHC15oentriesS<0.05","LHC15o", "LHC15oRFMean","LHC15oentriesS",None,doDraw=1)

In [ ]:
df.head(5)

In [ ]:
df.shape